In [1]:
import pandas as pd
from serpapi.google_search import GoogleSearch

# SerpAPI 설정
SERP_API_KEY = 'upy4ot4fi8JNZNCY0Tf1Urf1HnSlvkx9keFrIPql31wKG7q1HqTa3GO3fyLN'

def get_news_comments(query):
    params = {
        "engine": "google",
        "q": query,
        "api_key": SERP_API_KEY
    }

    search = GoogleSearch(params)
    results = search.get_dict()

    comments = []
    for result in results.get("news_results", []):
        if "snippet" in result:
            comments.append(result["snippet"])

    return pd.DataFrame(comments, columns=["comment"])


In [2]:
if __name__ == "__main__":
    news_query = "ndivia"
    # twitter_query = "#latestnewstopic"

    news_comments_df = get_news_comments(news_query)
    # related_tweets_df = get_related_tweets(twitter_query)

    print("News Comments DataFrame")
    print(news_comments_df.head())

    # print("\nRelated Tweets DataFrame")
    # print(related_tweets_df.head())

    # 데이터프레임을 CSV 파일로 저장
    # news_comments_df.to_csv("news_comments.csv", index=False)
    # related_tweets_df.to_csv("related_tweets.csv", index=False)

News Comments DataFrame
Empty DataFrame
Columns: [comment]
Index: []


In [3]:
import pandas as pd
from serpapi.google_search import GoogleSearch

# SerpAPI 설정
SERP_API_KEY = 'c72bbdc6fe07a3a34105b7f46b7bed2b30f10eedf2e4fdfd92a9e49017121d9c'

query = 'NDIVIA'

params = {
        "engine": "google",
        "q": query,
        "api_key": SERP_API_KEY
    }

search = GoogleSearch(params)
results = search.get_dict()

In [4]:
results

{'search_metadata': {'id': '66a25f25c9990342197db212',
  'status': 'Success',
  'json_endpoint': 'https://serpapi.com/searches/010bd3dcf68f3947/66a25f25c9990342197db212.json',
  'created_at': '2024-07-25 14:20:21 UTC',
  'processed_at': '2024-07-25 14:20:21 UTC',
  'google_url': 'https://www.google.com/search?q=NDIVIA&oq=NDIVIA&sourceid=chrome&ie=UTF-8',
  'raw_html_file': 'https://serpapi.com/searches/010bd3dcf68f3947/66a25f25c9990342197db212.html',
  'total_time_taken': 1.77},
 'search_parameters': {'engine': 'google',
  'q': 'NDIVIA',
  'google_domain': 'google.com',
  'device': 'desktop'},
 'search_information': {'query_displayed': 'NDIVIA',
  'total_results': 650000000,
  'time_taken_displayed': 0.31,
  'organic_results_state': 'Some results for exact spelling but showing fixed spelling',
  'spelling_fix': 'NVIDIA',
  'showing_results_for': 'NVIDIA'},
 'knowledge_graph': {'title': 'Nvidia',
  'type': 'Software company',
  'entity_type': 'companies, chains, company',
  'kgmid': '/m

In [30]:
import requests
from bs4 import BeautifulSoup
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd

# 감정 분석기 초기화
analyzer = SentimentIntensityAnalyzer()

# Reddit 게시물 URL
reddit_url = 'https://www.reddit.com/r/NVDA_Stock/comments/1eb38is/nvidias_stock_correction_will_lead_to_outsized/?rdt=60045'

# HTTP 요청 헤더 설정 (웹 스크래핑을 위해 유저 에이전트 설정)
headers = {'User-Agent': 'Mozilla/5.0'}

# 페이지 가져오기
response = requests.get(reddit_url, headers=headers)
if response.status_code != 200:
    raise Exception(f"Error: Unable to access page (status code {response.status_code})")

# 페이지 파싱
soup = BeautifulSoup(response.text, 'html.parser')

# 댓글 추출
comments = soup.find_all('div', class_='md')

# 댓글과 감정 분석 결과 저장
comment_data = []

for comment in comments:
    comment_text = comment.get_text()
    sentiment = analyzer.polarity_scores(comment_text)
    comment_data.append({
        'comment': comment_text,
        'sentiment': sentiment
    })

# 데이터프레임으로 변환
df = pd.DataFrame(comment_data)

# 데이터프레임 출력
print(df)

                                             comment  \
0  \n\nInvest when others are fearful\n\n    The ...   
1                                                      

                                           sentiment  
0  {'neg': 0.086, 'neu': 0.829, 'pos': 0.085, 'co...  
1  {'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound...  


In [35]:
df

,comment,sentiment
0,\n\nInvest when others are fearful\n\n The ...,"{'neg': 0.086, 'neu': 0.829, 'pos': 0.085, 'co..."
1,,"{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound..."


In [ ]:
import time
from selenium import webdriver
from bs4 import BeautifulSoup
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd

# 감정 분석기 초기화
analyzer = SentimentIntensityAnalyzer()

# Reddit 게시물 URL
reddit_url = 'https://www.reddit.com/r/subreddit/comments/post_id/post_title/'

# Selenium 웹 드라이버 설정 (Chrome)
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # 브라우저가 보이지 않도록 설정
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
driver = webdriver.Chrome(options=options)

# 페이지 로드
driver.get(reddit_url)

# 페이지가 모두 로드될 때까지 대기 (최대 10초)
time.sleep(10)

# "load more comments" 버튼을 모두 클릭하여 모든 댓글을 로드
while True:
    try:
        load_more_button = driver.find_element_by_xpath('//button[contains(text(), "load more comments")]')
        load_more_button.click()
        time.sleep(5)  # 댓글이 로드될 시간을 줍니다
    except Exception as e:
        break  # 더 이상 "load more comments" 버튼이 없으면 종료

# 페이지 소스를 가져와서 BeautifulSoup으로 파싱
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Selenium 드라이버 종료
driver.quit()

# 댓글 추출
comments = soup.find_all('div', class_='md')

# 댓글과 감정 분석 결과 저장
comment_data = []

for comment in comments:
    comment_text = comment.get_text()
    sentiment = analyzer.polarity_scores(comment_text)
    comment_data.append({
        'comment': comment_text,
        'sentiment': sentiment
    })

# 데이터프레임으로 변환
df = pd.DataFrame(comment_data)

# 데이터프레임 출력
print(df)

# CSV 파일로 저장
df.to_csv('reddit_comments_sentiment.csv', index=False)